In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series,DataFrame
from numpy import nan as NaN


In [4]:
from datetime import datetime
start_time = datetime.now()
start_time

datetime.datetime(2022, 1, 17, 15, 38, 30, 437147)

In [5]:
#df_ABC_distance = pd.read_csv('data/distance_y2019_all.csv',sep=',')
df_ABC_distance = pd.read_excel('data/DZDB4.xlsx',sheet_name="Samples")

df_ABC_distance_100Ma = df_ABC_distance[np.logical_and(df_ABC_distance['Est. Depos. Age (Ma)'] >= 0 , df_ABC_distance['Est. Depos. Age (Ma)'] <= 100 )]

Sample_ID = df_ABC_distance_100Ma['Ref_Sample_Key']
Sample_ID = Sample_ID.to_list()

df_ABC_distance_100Ma = pd.DataFrame(df_ABC_distance_100Ma)
df_ABC_distance_100Ma.loc[:,'true_type'] = 'a'
df_true_type = DataFrame()
for i in Sample_ID:
    distance_subduction = df_ABC_distance_100Ma[df_ABC_distance_100Ma['Ref_Sample_Key'] == i]
    if 0 <= float(distance_subduction['distance_subduction']) < 500:
        distance_subduction = distance_subduction.replace('a','a')
        df_true_type = df_true_type.append(distance_subduction[:],ignore_index=True)
    elif float(distance_subduction['distance_subduction']) >= 500 and float(distance_subduction['distance_subduction']) <= 2000:
        distance_subduction = distance_subduction.replace('a','b')
        df_true_type = df_true_type.append(distance_subduction[:],ignore_index=True)
    else:
        distance_subduction = distance_subduction.replace('a','c')
        df_true_type = df_true_type.append(distance_subduction[:],ignore_index=True)


In [6]:
df_ABC_distance_100Ma['Est. Depos. Age (Ma)'].max()

100.0

In [8]:
df_Data = pd.read_excel('data/DZDB4.xlsx',sheet_name="UPb_Data")

SedimentaryZircon = pd.merge(df_true_type,df_Data,on='Ref_Sample_Key')
#SedimentaryZircon.columns

In [9]:
SedimentaryZircon.columns

Index(['Sample_No.', 'Ref_Number', 'Ref_Sample_Key',
       'Ref No. In_Source_Database', 'Sample No. In_Source_Database',
       'Source_Database_x', 'Published Sample_ID', 'Country/State', 'Region',
       'Continent', 'Major Geographic-Geologic Description',
       'Minor Geologic-Geographic Unit', 'Locality', 'Latitude', 'Longitude',
       'Max. Depos. Age (Ma)', 'Est. Depos. Age (Ma)', 'Min. Depos. Age (Ma)',
       'Mineral', 'Mass Spectrometer', 'Spectrometer Location', 'Institution',
       'Spectrometer Model', 'Class-1 Rock Type', 'Class-2 Rock Type',
       'Class-3 Rock Type', 'Felsic/Mafic', 'Max Latitude', 'Min Latitude',
       'Max Longitude', 'Min Longitude', 'U-Pb Record Count',
       'U-Pb Record Count Standard_Model_Age', 'type', 'colour', 'FROMAGE',
       'TOAGE', 'distance_subduction', 'distance_subduction_m2016',
       'distance_Sengor_Natalin_rifts', 'distance_passive_margin_rifts',
       'distance_SN_PM_rifts', 'distance_active', 'distance_inactive',
     

In [14]:
SedimentaryZircon_simplify = SedimentaryZircon[['Ref_Sample_Key', 'Est. Depos. Age (Ma)', 'Standard_Model_Age', 'true_type']]
df_true_type_simplify = df_true_type[['Ref_Sample_Key', 'true_type']]


# better threshold test

In [16]:
#value1 = np.arange(0,1.01,0.01)
#value2 = np.arange(0,3010,10)

value1 = np.arange(0,0.11,0.01)
value2 = np.arange(0,100,10)
list_threshold = []

Sample_ID = df_true_type_simplify['Ref_Sample_Key']
Sample_ID = Sample_ID.to_list()
for iii in value1:
    iii=round(iii,2)
    for ii in value2:
        df_try = DataFrame()
        for i in Sample_ID:
            Sample_age = SedimentaryZircon_simplify[SedimentaryZircon_simplify['Ref_Sample_Key'] == i]

            age_list = Sample_age['Standard_Model_Age']
            CA_DA = age_list - Sample_age['Est. Depos. Age (Ma)']            
            CA_DA_Order = CA_DA.sort_values(ascending = True)
            
            sample_plot = df_true_type_simplify[df_true_type_simplify['Ref_Sample_Key'] == i]
            sample_plot = sample_plot.copy()
            if CA_DA_Order.iloc[int(len(CA_DA)*iii)] <= ii:
                sample_plot.loc[:,'test_type'] = 'A'
                df_try = df_try.append(sample_plot[:],ignore_index=True)
            else:
                sample_plot.loc[:,'test_type'] = 'B or C'
                df_try = df_try.append(sample_plot[:],ignore_index=True)

        PP = df_try[df_try['test_type'] == 'A']
        TP = PP[PP['true_type'] == 'a']
        FP = PP[PP['true_type'] != 'a']
        PN = df_try[df_try['test_type'] != 'A']
        TN = PN[PN['true_type'] != 'a']
        FN = PN[PN['true_type'] == 'a']
        
        list_sub = [iii,ii,len(PP),len(TP),len(FP),len(PN),len(TN),len(FN)]
        
        list_threshold.append(list_sub)
        

In [19]:
print(len(list_threshold))
print(list_threshold)

110
[[0.0, 0, 473, 298, 175, 2793, 1909, 884], [0.0, 10, 1267, 691, 576, 1999, 1508, 491], [0.0, 20, 1593, 801, 792, 1673, 1292, 381], [0.0, 30, 1836, 878, 958, 1430, 1126, 304], [0.0, 40, 1988, 943, 1045, 1278, 1039, 239], [0.0, 50, 2061, 984, 1077, 1205, 1007, 198], [0.0, 60, 2126, 1016, 1110, 1140, 974, 166], [0.0, 70, 2179, 1031, 1148, 1087, 936, 151], [0.0, 80, 2238, 1052, 1186, 1028, 898, 130], [0.0, 90, 2322, 1071, 1251, 944, 833, 111], [0.01, 0, 349, 232, 117, 2917, 1967, 950], [0.01, 10, 1102, 630, 472, 2164, 1612, 552], [0.01, 20, 1424, 748, 676, 1842, 1408, 434], [0.01, 30, 1674, 839, 835, 1592, 1249, 343], [0.01, 40, 1830, 905, 925, 1436, 1159, 277], [0.01, 50, 1921, 948, 973, 1345, 1111, 234], [0.01, 60, 1978, 977, 1001, 1288, 1083, 205], [0.01, 70, 2028, 996, 1032, 1238, 1052, 186], [0.01, 80, 2093, 1019, 1074, 1173, 1010, 163], [0.01, 90, 2174, 1038, 1136, 1092, 948, 144], [0.02, 0, 234, 163, 71, 3032, 2013, 1019], [0.02, 10, 907, 547, 360, 2359, 1724, 635], [0.02, 20, 1

In [ ]:
df_threshold=DataFrame(list_threshold)
df_threshold.rename(columns={0:'y',1:'x',2:'PP',3:'TP',4:'FP',5:'PN',6:'TN',7:'FN'},inplace=True)
df_threshold

In [ ]:
outputpath='data/better_threshold.csv'
df_threshold.to_csv(outputpath,sep=',',index = False,header = True)

In [2]:
df_threshold = pd.read_csv('data/better_threshold.csv',sep=',')

# PS: sensitivity = recall = TPR

In [12]:
for i in df_threshold.index:
    df_threshold.loc[i,'sensitivity'] = df_threshold.loc[i,'TP']/(df_threshold.loc[i,'TP']+df_threshold.loc[i,'FN'])
    df_threshold.loc[i,'specificity'] = df_threshold.loc[i,'TN']/(df_threshold.loc[i,'TN']+df_threshold.loc[i,'FP'])
    df_threshold.loc[i,'precision'] = df_threshold.loc[i,'TP']/(df_threshold.loc[i,'TP']+df_threshold.loc[i,'FP'])
    df_threshold.loc[i,'recall'] = df_threshold.loc[i,'TP']/(df_threshold.loc[i,'TP']+df_threshold.loc[i,'FN'])
    df_threshold.loc[i,'accuracy'] = (df_threshold.loc[i,'TP']+df_threshold.loc[i,'TN'])/(df_threshold.loc[i,'TP']+df_threshold.loc[i,'FP']+df_threshold.loc[i,'TN']+df_threshold.loc[i,'FN'])   
    df_threshold.loc[i,'FPR'] = df_threshold.loc[i,'FP']/(df_threshold.loc[i,'FP']+df_threshold.loc[i,'TN'])
    df_threshold.loc[i,'TPR'] = df_threshold.loc[i,'TP']/(df_threshold.loc[i,'TP']+df_threshold.loc[i,'FN'])
    df_threshold.loc[i,'F1'] = 2*df_threshold.loc[i,'precision']*df_threshold.loc[i,'recall']/(df_threshold.loc[i,'precision']+df_threshold.loc[i,'recall'])
    df_threshold.loc[i,'Fβ2'] = 5*df_threshold.loc[i,'precision']*df_threshold.loc[i,'recall']/(4*df_threshold.loc[i,'precision']+df_threshold.loc[i,'recall'])
    df_threshold.loc[i,'Fβ0.5'] = 1.25*df_threshold.loc[i,'precision']*df_threshold.loc[i,'recall']/(0.5*df_threshold.loc[i,'precision']+df_threshold.loc[i,'recall'])
    df_threshold.loc[i,'specificity'] = df_threshold.loc[i,'TN']/(df_threshold.loc[i,'FP']+df_threshold.loc[i,'TN'])

#print(df_threshold[df_threshold['accuracy'] == df_threshold['accuracy'].max()])
#print(df_threshold[df_threshold['precision'] == df_threshold['precision'].max()])
#print(df_threshold[df_threshold['recall'] == df_threshold['recall'].max()])
#print(df_threshold[df_threshold['F1'] == df_threshold['F1'].max()])
#print(df_threshold[df_threshold['Fβ2'] == df_threshold['Fβ2'].max()])
#print(df_threshold[df_threshold['Fβ0.5'] == df_threshold['Fβ0.5'].max()])

outputpath='data/better_threshold.csv'
df_threshold.to_csv(outputpath,sep=',',index = False,header = True)